In [1]:
#Download arxiv data through kaggle CLI or from semantics scholar with fetch_database.ipynb
#(For arXiv, make sure your phone and identity have been verified in your profile before dowloading the access token from kaggle
# then run the CLI -> kaggle datasets download -d Cornell-University/arxiv -p destination_folder --unzip

In [4]:
import sys
sys.path.append("..")
sys.path.append("../streamlit")

import pandas as pd

metadata_sample = "../streamlit/data/metadata_2025.parquet"
textdata_sample = "../streamlit/data/text_2025.parquet"

meta_df = pd.read_parquet(metadata_sample).head(100)
text_df = pd.read_parquet(textdata_sample).head(100)

meta_df.to_csv("../streamlit/data/metadata_sample_2025.csv")
text_df.to_csv("../streamlit/data/textdata_sample_2025.csv")

In [1]:
import os
import sys

#Import config file. Update config.py according to your environment
import config

import pandas as pd
import numpy as np
import h5py

import requests, json

from transformers import AutoTokenizer, AutoModel
from adapters import AutoAdapterModel
import torch
import torch.nn.functional as F

from tqdm import tqdm
import time

In [2]:
dataset_source = 'semanticsscholar' #'arxiv'

## 1. Load dataset

In [3]:
if 'arxiv' in dataset_source.lower():
    data_path = os.path.join(config.path_to_data, 'arxiv-metadata-oai-snapshot.json')
    data = pd.read_json(data_path, lines=True)
elif 'semantics' in dataset_source.lower():
    data_path = os.path.join(config.path_to_data, 'semnaticsscholar_')
    metadata = pd.read_parquet(data_path + "metadata.parquet", engine="pyarrow")
    textdata = pd.read_parquet(data_path + "text.parquet", engine="pyarrow")

display(metadata)
display(textdata)

,paperId,openAccessPdf,fieldsOfStudy,venue,authorName,authorId,citationCount,influentialCitationCount,publicationDate,publicationYear
0,0001d5fbff6f7c763a78dd5d141292416dcfae59,https://aacrjournals.org/clincancerres/article...,"Biology,Medicine",Clinical Cancer Research,"T. Ishibe,T. Nakayama,T. Okamoto,T. Aoyama,K. ...","14839966,10239302,2000448727,3201672,7147569,2...",88,6.0,2005-04-01,2005
1,000372290caae5482dcdd7954feb895ccbab921d,http://www.merl.com/publications/docs/TR2005-0...,"Computer Science,Mathematics",IEEE Transactions on Communications,"Juntan Zhang,M. Fossorier","3193350,48891339",319,26.0,2005-03-07,2005
2,000376ec6f6ade6261fc12df9eddd25a88c1e9ca,https://www.jstage.jst.go.jp/article/bpb/28/8/...,Medicine,Biological and Pharmaceutical Bulletin,"Bi-qi Zhang,Shen-Jiang Hu,Lihong Qiu,Q. Shan,J...","11809650,6617669,123635715,11305673,2152146096...",24,2.0,2005-08-01,2005
3,0003a5747ee3676be1b6d2c72c13a04e41811274,https://onlinelibrary.wiley.com/doi/pdfdirect/...,"Biology,Medicine",Developmental Dynamics,"Barden Chan,S. Sinha,Dan Cho,R. Ramchandran,V....","35690923,2114125286,2113252574,2134773,3658233",53,2.0,2005-01-01,2005
4,00043e28cffe744909bf69cde70e6dac0bce0a10,http://www.jbc.org/article/S0021925820589394/pdf,"Chemistry,Medicine",Journal of Biological Chemistry,"M. Westfall,A. M. Lee,D. Robinson","5025983,49249911,48706510",70,3.0,2005-12-16,2005
...,...,...,...,...,...,...,...,...,...,...
3737188,fea6042ae3d4b2e873426e75ef4e3908fefdf2e7,http://arxiv.org/pdf/2501.11288,Computer Science,IEEE transactions on consumer electronics,"Yanchao Wang,Dawei Zhang,Run Li,Zhonglong Zhen...","2298569900,2275025303,2298564302,2278244261,23...",0,0.0,2025-01-20,2025
3737189,fed4e1c03728d1747984ccc035e9e52d4f576666,https://www.biorxiv.org/content/biorxiv/early/...,Biology,bioRxiv,"Niko Kasalo,Mirjana Domazet-Lošo,Tomislav Doma...","2145558181,1414104312,1397755049",2,0.0,2025-01-16,2025
3737190,fee975b0352fbf35fdc1926b25a16d8531ebb4e6,https://www.researchsquare.com/article/rs-4129...,Computer Science,Multim. Syst.,"Liang Yang,Qi Yang,Jingjie Zeng,Tao Peng,Zhiha...","2143920912,2308550856,2000361118,2068930271,15...",0,0.0,2025-02-06,2025
3737191,ff8fd5f30461bb788c6a32f37fe0f7821eafebf6,http://arxiv.org/pdf/2501.00799,"Computer Science,Mathematics",IEEE Signal Processing Letters,"S. Mukhopadhyay,Debasmita Mukherjee","46287665,2338269614",0,0.0,2025-01-01,2025


,paperId,title,abstract,openAccessPdf
0,0001d5fbff6f7c763a78dd5d141292416dcfae59,Disruption of Fibroblast Growth Factor Signal ...,Purpose: Synovial sarcoma is a soft tissue sar...,https://aacrjournals.org/clincancerres/article...
1,000372290caae5482dcdd7954feb895ccbab921d,Shuffled iterative decoding,Shuffled versions of iterative decoding of low...,http://www.merl.com/publications/docs/TR2005-0...
2,000376ec6f6ade6261fc12df9eddd25a88c1e9ca,Diphasic effects of Astragalus membranaceus BU...,This study was designed to investigate the eff...,https://www.jstage.jst.go.jp/article/bpb/28/8/...
3,0003a5747ee3676be1b6d2c72c13a04e41811274,Critical roles of CD146 in zebrafish vascular ...,"In this report, we use zebrafish as a model sy...",https://onlinelibrary.wiley.com/doi/pdfdirect/...
4,00043e28cffe744909bf69cde70e6dac0bce0a10,Differential Contribution of Troponin I Phosph...,Cardiac troponin I is a phosphorylation target...,http://www.jbc.org/article/S0021925820589394/pdf
...,...,...,...,...
3737188,fea6042ae3d4b2e873426e75ef4e3908fefdf2e7,PD-SORT: Occlusion-Robust Multi-Object Trackin...,Multi-object tracking (MOT) is a rising topic ...,http://arxiv.org/pdf/2501.11288
3737189,fed4e1c03728d1747984ccc035e9e52d4f576666,Massive outsourcing of energetically costly am...,Animals are generally capable of synthesizing ...,https://www.biorxiv.org/content/biorxiv/early/...
3737190,fee975b0352fbf35fdc1926b25a16d8531ebb4e6,Dialogue sentiment analysis based on dialogue ...,None,https://www.researchsquare.com/article/rs-4129...
3737191,ff8fd5f30461bb788c6a32f37fe0f7821eafebf6,Follow the Approximate Sparse Leader for No-Re...,We consider the problem of online sparse linea...,http://arxiv.org/pdf/2501.00799


## 2- Load embedding model and compute article embeddings

In [20]:
model_name = 'allenai/specter2'#'sentence-transformers/all-mpnet-base-v1' #
batch_size = 128
max_length = 512
Nsamples = len(data)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
if 'sentence-transformer' in model_name:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
elif 'specter2' in model_name:
    tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')
    model = AutoAdapterModel.from_pretrained('allenai/specter2_base')
    model.load_adapter(model_name, source="hf", set_active=True)

model.to(device);
for param in model.parameters():
    param.requires_grad = False

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
# preprocess the input
embeddings = []

model.eval()
with torch.no_grad():
    with tqdm(range(Nsamples//batch_size + 1), desc=f'total of {Nsamples} papers', unit='batch') as pbar:
        for i in pbar:
            batch_text = data.iloc[i*batch_size:(i + 1)*batch_size]['title'] + tokenizer.sep_token + data.iloc[i*batch_size:(i + 1)*batch_size]['abstract']
            batch_text = batch_text.to_list()
            if batch_text:
                batch_tokens = tokenizer(batch_text, padding=True, truncation=True, return_tensors="pt", return_token_type_ids=False, max_length=max_length)
                for key in batch_tokens.keys():
                    batch_tokens[key] = batch_tokens[key].to(device)
                
                output = model(**batch_tokens)
                # first token in the batch as the embedding
                if 'pooler_output' in output.keys():
                    embeddings_batch = output.pooler_output.cpu().numpy().astype(np.float32)
                else:
                    embeddings_batch = output.last_hidden_state[:, 0, :].cpu().numpy().astype(np.float32)
                    
                embeddings.append(embeddings_batch)

total of 473037 papers:   0%|          | 0/3696 [00:00<?, ?batch/s]

total of 473037 papers: 100%|██████████| 3696/3696 [1:18:58<00:00,  1.28s/batch]


## 3- Save embeddings with metadata

In [23]:
model_nickname = model_name.split('/')[-1]
embeddings_filepath = os.path.join(config.path_to_data, 'embeddings', f'embeddings_{dataset_source}_{model_nickname}.h5')

In [24]:
embeddings = np.concat(embeddings, axis=0)

In [27]:
B, D = embeddings.shape
with pd.HDFStore(embeddings_filepath, mode="w", complib="blosc", complevel=9) as store:
    store.put("metadata", data, format="table", data_columns=True)
    store.put("embeddings", pd.DataFrame(embeddings).set_index(data.index), format="table")

## 4- Load embeddings and query model

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
max_length = 512
model_name = 'allenai/specter2'#'sentence-transformers/all-mpnet-base-v1' #
model_nickname = model_name.split('/')[-1]
embeddings_filepath = os.path.join(config.path_to_data, 'embeddings', f'embeddings_{dataset_source}_{model_nickname}.h5')
center_embeddings = False

if 'sentence-transformer' in model_name:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model_query = AutoModel.from_pretrained(model_name)
elif 'specter2' in model_name:
    tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')
    model_query = AutoAdapterModel.from_pretrained('allenai/specter2_base')
    model_query.load_adapter("allenai/specter2_adhoc_query", source="hf", set_active=True)

model_query.to(device);
for param in model_query.parameters():
    param.requires_grad = False

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
with pd.HDFStore(embeddings_filepath, mode="r") as store:
    data = store.select("metadata")#, where="citationCount > 10")
    idx_filtered = data.index.to_list()
    embeddings = store.select("embeddings", where=f"index in {idx_filtered}").to_numpy()

In [30]:
#normalize embeddings
if center_embeddings:
    embeddings_mean = embeddings.mean(axis=0, keepdims=True)
else:
    embeddings_mean = 0

embeddings_norm = (embeddings - embeddings_mean) / np.linalg.norm(embeddings - embeddings_mean,axis=-1, keepdims=True)

## 4- Ranking based on cos similarity

In [53]:
K = 10
query = "Gerneral review on usage of transformers in biology" #data.iloc[1200]['title']
#possibly parse, expand and reformat the query using a LLM

In [54]:
query_input = tokenizer([query], padding=True, truncation=True,
                              return_tensors="pt", return_token_type_ids=False, max_length=max_length)

for key in query_input.keys():
    query_input[key] = query_input[key].to(device)
with torch.no_grad():
    output = model_query(**query_input)
    if 'pooler_output' in output.keys():
        embs = output.pooler_output.cpu().numpy()
    else:
        embs = output.last_hidden_state[:, 0, :].cpu().numpy()

#normalize embedding
embs_norm = (embs - embeddings_mean) / np.linalg.norm(embs - embeddings_mean,axis=-1, keepdims=True)

In [55]:
#Compute cos similarity and find topK
similarity_matrix = np.einsum('bc,kc->bk', embs_norm, embeddings_norm)
topK_indices = np.argsort(-similarity_matrix, axis=-1)[:,:K]
topK_values = np.take_along_axis(similarity_matrix, topK_indices, axis=-1)

topK_indices = topK_indices[0]
topK_values = topK_values[0]

In [56]:
print(topK_indices,'\n',topK_values)

[407009  38751 408477  13335  95296 112836 364519  48757 136316 467049] 
 [0.8869703  0.88259465 0.88202095 0.8807894  0.880435   0.8800655
 0.8794871  0.87924373 0.8786969  0.87815005]


In [57]:
for k in topK_indices[:10]:
    print(k)
    print(data.iloc[k]['title'])
    print(data.iloc[k]['abstract'])

407009
Research on General-Purpose Brain-Inspired Computing Systems
None
38751
Physical deep learning with biologically inspired training method: gradient-free approach for physical hardware
None
408477
A survey on dynamic populations in bio-inspired algorithms
None
13335
A computational model of organism development and carcinogenesis resulting from cells’ bioelectric properties and communication
None
95296
A Curve Approximation Approach Using Bio-inspired Polar Coordinate Bald Eagle Search Algorithm
None
112836
Hopf physical reservoir computer for reconfigurable sound recognition
None
364519
Bioimpedance basics and phase angle fundamentals
None
48757
Efficiency analysis of artificial vs. Spiking Neural Networks on FPGAs
None
136316
Dynamic analysis and application in medical digital image watermarking of a new multi-scroll neural network with quartic nonlinear memristor
None
467049
Research on the pathogenesis of Alzheimer's disease based on thalamocortical computational model
Alpha 

## 5- Refinement with recommendation model or based on user previous selection

## 5- RAG based on topK search

In [ ]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [58]:
abstracts = '\n\n'.join(data.iloc[topK_indices]['abstract'].to_list())
messages = [
    {"role": "system", "content": "You are a research assistant. Answer the user’s query as accurately as possible using your knowledge and the provided abstracts. \
     Do not summarize the abstracts unless explicitly asked. Use your knowledge first if the question is general. Only use the abstracts if this is explicitly asked"},
    {"role": "user", "content": f"Here are the abstracts of recent papers:\n\n{abstracts}\n\nBased on this information and on your knowledge, please answer the following question:\n\n{query}"}
]
outputs = pipe(
    messages,
    max_new_tokens=10000,
)
print(outputs[0]["generated_text"][-1]['content'])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on my general knowledge, I can provide a general overview of the usage of transformers in biology.

Transformers, a type of neural network architecture, have gained significant attention in recent years due to their effectiveness in various natural language processing (NLP) tasks. However, their applications extend beyond NLP, and they have been increasingly used in biology to analyze and interpret large biological datasets.

In biology, transformers are used for various purposes, including:

1. **Genomic analysis**: Transformers can be used to analyze genomic data, such as gene expression, mutation, and variation, to identify patterns and relationships that may not be apparent through traditional methods.
2. **Protein structure prediction**: Transformers can be used to predict the three-dimensional structure of proteins, which is crucial for understanding their function and behavior.
3. **Gene regulation analysis**: Transformers can be used to analyze the regulatory interactions

## Attempt to use Llama-3.2-70B locally

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "meta-llama/Llama-3.3-70B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

quantized_model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map='auto', torch_dtype=torch.bfloat16, quantization_config=quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)
input_text = "What are we having for dinner?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

output = quantized_model.generate(**input_ids, max_new_tokens=10)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What are we having for dinner? I was thinking we could go to that new restaurant


In [16]:
# Convert messages to ChatML format
abstracts = '\n\n'.join(data.iloc[topK_indices]['abstract'].to_list())
input_text = f"You are a research assistant. Use the provided research papers to answer questions accurately. Do not summarize unless asked." + f"Here are some research abstracts:\n\n{abstracts}\n\n Based on these, please answer the question: {query}\n"

# Tokenize and generate response
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
output = quantized_model.generate(**input_ids, max_new_tokens=200)

# Decode and print output
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


you are a research assistant. use the provided research papers to answer questions accurately. do not summarize unless asked. here are some research abstracts : we propose a new approach to learned optimization where we represent the computation of an optimizer ' s update step using a neural network. the parameters of the optimizer are then learned by training on a set of optimization tasks with the objective to perform minimization efficiently. our innovation is a new neural network architecture, optimus, for the learned optimizer inspired by the classic bfgs algorithm. as in bfgs, we estimate a preconditioning matrix as a sum of rank - one updates but use a transformerbased neural network to predict these updates jointly with the step length and direction. in contrast to several recent learned optimization - based approaches [ 24, 27 ], our formulation allows for conditioning across the dimensions of the parameter space of the target problem while remaining applicable to optimization

In [46]:
import gc
gc.collect()
torch.cuda.empty_cache()